# TP n°4 - Feature selection
**Objectif:**

Ce TP vise à étudier quelques méthodes de sélection de variables (Filtrage) en utilisant la bibliothèque Scikit-learn.

[Documentation scikit-learn -> feature selection](https://scikit-learn.org/stable/modules/feature_selection.html)

Durant ce TP, on va aussi utiliser un modèle de réduction de variables (AFD).
Dans Scikit-learn, l'analyse factorielle discriminante (AFD) est mise en oeuvre dans la classe [LinearDiscriminantAnalysis](http://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html)

# Scikit-learn
Scikit-learn ou sklearn est une bibliothèque python libre conçue pour effectuer de l'apprentissage automatique.
Elle propose un set d'algorithmes de classification, régression et regroupement etc.

# Dataset
Pour illustrer le propos, on va utiliser le dataset `sales.csv` et nous allons ajouter des variables aléatoires qui représentent du bruit.


In [63]:
# Chargement du dataset sales.csv
import numpy as np
import pandas as pd

df = pd.read_csv('../datasets/sales.csv')
df.head(2)

,division,level of education,training level,work experience,salary,sales
0,printers,associate's degree,0,7,82712,307122
1,printers,some college,2,3,75645,221423


In [64]:
# Dimension du dataset
print(df.shape)

(1000, 6)


Pour l'exemple, on va supposer que la variable `level of education` (variable expliquée) contient les étiquettes des classes.

Pour référence, appliquons l'étape décisionnelle de l'analyse discriminante (comme modèle décisionnel) sur les données initiales et ensuite sur les données auxquelles les nouvelles variables aléatoires ont été ajoutées :

In [65]:
# Préparation des subsets d'entrainement et de test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encoding des variables catégorielles
label_encoder = LabelEncoder()
df['division labels'] = label_encoder.fit_transform(df['division'])
df['education labels'] = label_encoder.fit_transform(df['level of education'])

# Ajout de 2 variables pour bruiter
np.random.seed(42)
df['var1'] = np.random.uniform(0, 1, size=len(df)) # loi uniforme
df['var2'] = np.random.randn(len(df), 1) # loi normale

df.tail(2)

,division,level of education,training level,work experience,salary,sales,division labels,education labels,var1,var2
998,computer hardware,some college,1,7,97445,354507,0,4,0.950237,-0.429302
999,peripherals,some college,2,3,76189,307107,3,4,0.446006,-0.692421


In [66]:
X = df.drop(['level of education', 'education labels', 'division'], axis=1) # variables explicatives
y = df['education labels'] # variable expliquée
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state=42)
print(f"dimension du subset d'entrainement: {X_train.shape}")
print(f"dimension du subset de test: {X_test.shape}")

dimension du subset d'entrainement: (500, 7)
dimension du subset de test: (500, 7)


In [67]:
X_train[0:2]

,training level,work experience,salary,sales,division labels,var1,var2
680,1,6,85795,307952,0,0.817072,-0.138456
177,2,10,112332,488069,0,0.386735,0.456753


In [68]:
# Création du modèle
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model = LinearDiscriminantAnalysis()

# données initiales
X_train_org = X_train.iloc[:,0:5]
X_test_org = X_test.iloc[:,0:5]

model.fit(X_train_org, y_train)
print(f"accuracy on the original test subset: {model.score(X_test_org, y_test):.2f}")

# entrainement avec les données bruitées
model.fit(X_train, y_train)
print(f"accuracy on the modified test subset: {model.score(X_test, y_test):.2f}")

accuracy on the original test subset: 0.49
accuracy on the modified test subset: 0.49


### Question
Comparer les performances du modèle entrainé sur les données initiales puis sur les données bruitées.

### Correction
On peut constater que la présence des nouvelles variables, de valeurs aléatoires, a un impact négatif sur la capacité de généralisation du modèle décisionnel construit.

# Filtrage
La famille de filtrage est appliquée sans faire appel à un modèle prédictif.
Elle permet de maximiser l'information mutuelle entre la variable d'entrée et celle de sortie.
Elle minimise la redondance entre les variables d'entrée.
NB: Une coopération sous optimale avec le modèle prédictif qui n'intervient pas dans la sélection.
Cette approche présente un coût inférieur à celui de l'approche Wrapper.

## Suppression de variables à faible variance
La suppression de variables à faible variance consiste à éliminer les variables dont la variance est inférieure à un seuil (par défaut la valeur du seuil est 0, sont donc éliminées les variables constantes).
[Voir plus](https://scikit-learn.org/stable/modules/feature_selection.html#removing-features-with-low-variance)

In [69]:
X_train.iloc[0:3]

,training level,work experience,salary,sales,division labels,var1,var2
680,1,6,85795,307952,0,0.817072,-0.138456
177,2,10,112332,488069,0,0.386735,0.456753
395,2,4,77729,308041,2,0.930757,-0.280675


In [70]:
y_train.iloc[0:3]

680    0
177    2
395    0
Name: education labels, dtype: int64

In [71]:
idx = [680, 177, 395]
df.iloc[idx]

,division,level of education,training level,work experience,salary,sales,division labels,education labels,var1,var2
680,computer hardware,associate's degree,1,6,85795,307952,0,0,0.817072,-0.138456
177,computer hardware,high school,2,10,112332,488069,0,2,0.386735,0.456753
395,office supplies,associate's degree,2,4,77729,308041,2,0,0.930757,-0.280675


In [72]:
# Exemple de filtrage avec la suppression de variables à faible variance
from sklearn.feature_selection import VarianceThreshold

variance_threshold = VarianceThreshold(threshold=0)
Xtrain_removed0variance = variance_threshold.fit_transform(X_train)
Xtrain_removed0variance[0:3]

array([[ 1.00000000e+00,  6.00000000e+00,  8.57950000e+04,
         3.07952000e+05,  0.00000000e+00,  8.17072071e-01,
        -1.38455984e-01],
       [ 2.00000000e+00,  1.00000000e+01,  1.12332000e+05,
         4.88069000e+05,  0.00000000e+00,  3.86735346e-01,
         4.56753219e-01],
       [ 2.00000000e+00,  4.00000000e+00,  7.77290000e+04,
         3.08041000e+05,  2.00000000e+00,  9.30757326e-01,
        -2.80675077e-01]])

In [73]:
X_train[0:3]

,training level,work experience,salary,sales,division labels,var1,var2
680,1,6,85795,307952,0,0.817072,-0.138456
177,2,10,112332,488069,0,0.386735,0.456753
395,2,4,77729,308041,2,0.930757,-0.280675


In [56]:
# affichage des variances des différentes variables
variance_threshold.variances_

array([9.30796000e-01, 8.67897600e+00, 1.04496000e+05, 6.74525000e+05,
       1.83865600e+00, 8.44071524e-02, 1.01471354e+00])

In [74]:
# Fixer un seuil pour éliminer quelques variables
variance_threshold10 = VarianceThreshold(threshold=10)
X_train_best1 = variance_threshold10.fit_transform(X_train)
X_train_best1[0:3]

array([[ 85795., 307952.],
       [112332., 488069.],
       [ 77729., 308041.]])

In [59]:
# affichage des variances des différentes variables
variance_threshold10.variances_

array([9.30796000e-01, 8.67897600e+00, 3.19821440e+08, 1.16987994e+10,
       1.83865600e+00, 8.44071524e-02, 1.01471354e+00])

In [60]:
# Les variables sélectionnées
print(variance_threshold10.get_support())

[False False  True  True False False False]


In [75]:
print(variance_threshold10.get_feature_names_out())

['salary' 'sales']


### Question
Commenter la variance des 3 variables ajoutées var1, var2 et var3. (pour `threshold=10`)
A travers une autre combinaison de variables, proposer une 2ème classification via le même modèle étudié `from sklearn.discriminant_analysis import LinearDiscriminantAnalysis`.
Commenter le résultat obtenu.

### Correction
Pour `threshold=10`, selon la réponse de la méthode `variance_threshold2.get_support()`, les 3 variables var1, var2 et var3 ont été éliminées.
Seules les 2 variables `sales` et `salary` ont été sélectionnées. Elles représentent les 2 meilleures variables respectives en terme d'importance de la variance.

Examinons l'effet de cette sélection :

In [45]:
model = LinearDiscriminantAnalysis()
model.fit(X_train_best1, y_train)
X_test_best1 = variance_threshold10.transform(X_test)
print(f"accuracy on the best1 test subset: {model.score(X_test_best1, y_test):.2f}")

accuracy on the best1 test subset: 0.47


- Malgré la sélection des 2 meilleures variables en terme de variance, le modèle n'aboutit pas à une classification meilleure.
- Néanmoins, avec cette sélection on a gardé presque le même niveau de performance 0.49 ~ 0.47
- La réduction du nombre de variables explicatives réduit la complexité du modèle.
- D'autres paramètres contribuent à une généralisation meilleure.
- Avec un seuil 0, seulement 1 variable ajoutée sur 3 est éliminée car les variances des 2 autres sont bien supérieures à 0.

## Sélection univariée
La sélection univariées cherche à déterminer (à travers des tests statistiques comme le test de `Chi2`) dans quelle mesure chaque variable d'entrée "explique" la variable de sortie; les variables les moins explicatives individuellement sont éliminées.

Cette méthode élimine les variables pour lesquelles les valeurs de l'information mutuelle avec la variable de sortie sont les plus faibles (c'est à dire, qui « expliquent » le moins bien la variable de sortie). Nous avons l'intention de garder la moitié des variables et utilisons donc la fonction `SelectKBest` (d'autres sont disponibles, voir la documentation).

[Doc. sélection univariée](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection)
[Doc. information mutuelle pour la classification](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html#sklearn.feature_selection.mutual_info_classif)

In [81]:
# Exemple de filtrage avec la sélection univariée
from sklearn.feature_selection import SelectKBest, f_oneway
kBestMutualInfo = SelectKBest(score_func=f_oneway, k=2) # aussi mutual_info_classif
X_train_best2 = kBestMutualInfo.fit_transform(X_train, y_train)

# variables conservées (celles avec True)
print(kBestMutualInfo.get_support())

[False False  True  True False False False]


In [82]:
print(kBestMutualInfo.get_feature_names_out())

['salary' 'sales']


In [80]:
X_train[:3]

,training level,work experience,salary,sales,division labels,var1,var2
680,1,6,85795,307952,0,0.817072,-0.138456
177,2,10,112332,488069,0,0.386735,0.456753
395,2,4,77729,308041,2,0.930757,-0.280675


Les variables ajoutées, ont bien été éliminées par cette méthode.

### Question
Décrire la performance du modèle décisionnel. Comparer aux résultats obtenus par la méthode précédente.

### Correction

In [83]:
model = LinearDiscriminantAnalysis()
model.fit(X_train_best2, y_train)

X_test_best2 = kBestMutualInfo.transform(X_test)
print(f"accuracy on the best2 test subset: {model.score(X_test_best2, y_test):.2f}")

accuracy on the best2 test subset: 0.47


Pour l'échatillon expérimenté (faible nombre d'observations), on ne peut pas bien saisir la différence de performance de généralisation du modèle entre les 2 méthodes. En plus on remarque une large différence dans la distribution des variables d'où la nécessité d'une standardisation.

# Wrapper
La famille Wrapper coopère directement avec le modèle prédictif.
Choix des variables qui maximisent les performances du modèle.
**Inconvénients :**
- Coût élevé
- Pas de justification théorique de la sélection des variables
- Incompréhension des relations de dépendances entre les variables.
- La procédure de sélection est spécifique au modèle utilisé.

## Sélection séquentielle
Un modèle décisionnel doit être développé sur chaque (sous-)ensemble candidat de variables d'entrée et c'est la performance du modèle qui caractérise le (sous-)ensemble de variable. Deux versions sont proposées, une incrémentale (*Forward-SFS*) et une décrémentale (*Backward-SFS*).
[Voir plus](https://scikit-learn.org/stable/modules/feature_selection.html#sequential-feature-selection)

## Elimination récursive de variables
**Principe:** apprentissage du modèle sur la totalité des variables d'entrée pour extraire la pertinence de chaque variable (variance).
[Voir plus](https://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination)

## Sélection avec SelectFromModel
**Principe:** apprentissage du modèle sur la totalité des variables d'entrée pour extraire la pertinence de chaque variable (variance).
[Voir plus](https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection-using-selectfrommodel)

# Embedding
L'embedding (ou l'intégration) est une opération de sélection qui est intégrée à la méthode de construction de modèle. Pas de sur-coût par rapport à la construction du modèle mais cette approche ne peut pas être utilisée avec tout type de modèle.


# Exercice
Quelles sont les meilleures caractéristiques du dataset `sales.csv` qui permettent de prédire au mieux le salaire ?
**Note:** Vous pouvez utiliser ce modèle: `from sklearn.linear_model import LinearRegression`


# TP suivant ?
TP n°6: La réduction des variables par réduction des dimensions --> ACP.
